<span style="font-size:200%">MCMC</span><br>
<span style="color: gray">dic 2019</span><br>
[*Alberto Ruiz*](http://dis.um.es/profesores/alberto)

Las técnicas **Markov-Chain-Monte-Carlo** permiten obtener muestras de una densidad de probabilidad cuando solo es posible evaluar una función proporcional a dicha densidad (normalmente porque es intratable normalizarla). Esto es suficiente para resolver el problema de la inferencia Bayesiana mediante simulación.

## Introducción

En muchas situaciones los datos observables $D$ dependen de los parámetros $\theta$ de un modelo siguiendo una ley conocida:

$$p(D \,|\, \theta )$$

El problema fundamental de la inferencia probabilística es obtener información acerca los parámetros $\theta$ cuando se han observado unos datos concretos $D_o$. Es decir, buscamos la probabilidad condicionada contraria:

$$p( \theta \,|\, D_{o} )$$

Los parámetros $\theta$ son inicialmente desconocidos, aunque normalmente disponemos de cierta información $p(\theta)$, "a priori", sobre ellos.

Por tanto, estamos interesados en

$$p( \theta \,|\, D_{o} ) = \frac{p(D_o,\theta)}{p(D_o)} = \frac{p(D_o,\theta)}{\sum_\theta p(D_o,\theta)} \propto p(D_o,\theta)$$

Es decir, la distribución *a posteriori* es proporcional a la conjunta, evaluada en los datos observados, y donde quedan como variables los parámetros del modelo. La distribución conjunta es el producto del modelo de medida y la información a priori:

$$p(D_o,\theta) = p(D_o\mid\theta)\; p(\theta)$$

Por tanto

$$p( \theta \mid D_{o} ) \propto p( D_o \mid \theta ) \; p(\theta)$$




La información sobre los parámetros se expresa a su vez mediante hiperparámetros sobre los que se tiene de nuevo cierta información, y así sucesivamente.

$$P(D,\theta, \alpha) =  P(D\mid\theta,\alpha)\;p(\theta,\alpha) = P(D\mid\theta)\; p(\theta \mid \alpha)\; p(\alpha) $$

En definitiva, la densidad conjunta se expresa como un producto de densidades condicionadas explotando las dependencias del modelo.



Cuando el número de variables es pequeño se pueden aplicar técnicas de *grid*, recorriendo exhaustivamente el espacio de parámetros. (De hecho, los ejemplos de este notebook se pueden resolver fácilmente de esta forma.)

Pero en problemas interesantes esta cadena de factorizaciones puede ser bastante compleja, con una constante de normalización computacionalmente intratable. En algunas aplicaciones puede bastar con encontrar el parámetro más probable. Pero normalmente interesa también la incertidumbre de la estimación, lo que implica analizar regiones más amplias del espacio de parámetros.


Las técnicas MCMC permiten muestrear eficientemente cualquier densidad sin necesidad de que esté normalizada. Podemos aplicarlo a factorizaciones del tipo $P(D_o\mid\theta)\;p(\theta \mid \alpha)\; p(\alpha)$, y utilizar las muestras para extraer información sobre $p(\theta \mid D_o)$, marginalizando los parámetros auxiliares ("nuisance") que no nos interesen.

$$P(D,\theta) =  \sum_\alpha\; P(D,\theta,\alpha)$$

Esto se consigue de forma inmediata ignorando los valores de $\alpha$ en la muestra obtenida.

Estas [transparencias de Lam](http://pareto.uab.es/mcreel/IDEA2017/Bayesian/MCMC/mcmc.pdf) lo explican con más detalle.

## Implementación del algoritmo *Metropolis*

Para trabajar en serio es recomendable utilizar paquetes profesionales como [stan](http://mc-stan.org/) o [pymc3](http://docs.pymc.io/).

Sin embargo, para familizarizarnos con la técnica podemos experimentar con una implementación sencilla de la variante más simple: el algoritmo de [Metropolis](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm).

El objetivo es obtener muestras de $p(x)$ pero solo podemos evaluar $f(x)=k\; p(x)$, donde $k$ se desconoce. Dada la muestra actual $x_{k}$ generamos la muestra siguiente $x_{k+1}$ de acuerdo con la siguiente receta: generamos una perturbación $x_p$ alrededor de $x_k$ y calculamos el ratio de verosimilitudes $\rho =\min( f(x_p) / f(x_k), 1)$. Aceptamos $x_p$ como muestra siguiente con probabilidad $\rho$, y si la rechazamos repetimos la muestra anterior.

### Código

In [ ]:
import numpy as np

# paso elemental
def metropolisGen(sigma, x0, lprob):
    s = (np.array(x0), lprob(x0), True)
    while True:
        yield s
        xa, la, _ = s
        delta = sigma * np.random.randn(len(xa))
        x = xa + delta
        l = lprob(x)
        ratio = l - la
        accept = ratio > 0 or np.log(np.random.rand()) < ratio
        if accept:
            s = (x,l,True)
        else:
            s = (xa,la,False)

In [ ]:
from itertools import islice
from sys import stderr

# generación de muestras
def metropolis(lprob, n, burn, step, sigma, x0):
    run    = metropolisGen(sigma,x0,lprob)
    select = islice(run , burn , burn+n*step , step)
    sample, accept = zip(*[(s,a) for s,_,a in select])
    print('ρ = {:.3f}'.format(np.mean(accept)), file=stderr)
    return np.array(sample)

A continuación comprobamos su funcionamiento en un par de ejemplos sencillos.

### Muestreo de una normal 2D

La forma razonable de hacerlo es el método directo:

In [ ]:
mu  = np.array([1,1])
cov = np.array([[1,   0.8],
                [0.8, 1  ]])

In [ ]:
x,y = np.random.multivariate_normal(mu,cov,50).T

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(x,y,'.'); plt.axis('equal');

Vamos a comprobar si el algoritmo de Metropolis consigue un resultado similar.

In [ ]:
def lgauss(m, c):
    ic = np.linalg.inv(c)
    return lambda x: -0.5* (x-m) @ ic @ (x-m)     # + k

In [ ]:
test1 = metropolis(lgauss(mu,cov), n=100, burn=0, step=1, sigma=1, x0=[-3,3])

In [ ]:
plt.figure(figsize=(6,6))
x,y = test1.T
plt.plot(x,y,'.-',markersize=8,lw=0.5); plt.axis('equal');

In [ ]:
test2 = metropolis(lgauss(mu,cov), n=1000, burn=100, step=5, sigma=1, x0=[-3,3])

In [ ]:
plt.figure(figsize=(6,6))
x,y = test2.T
plt.plot(x,y,'.',markersize=8, alpha=0.3); plt.axis('equal');

In [ ]:
test2.mean(axis=0)

In [ ]:
np.cov(test2.T)

### Muestreo de una densidad toroidal

In [ ]:
lg = lgauss([2],[[0.3**2]])

test3 = metropolis(lambda v: lg(np.linalg.norm(v)), n=300, burn=1000, step=5, sigma=1, x0=[-3,3],)

In [ ]:
plt.figure(figsize=(6,6))
x,y = test3.T
plt.plot(x,y,'.',markersize=8); plt.axis('equal');

### Justificación intuitiva

Consideremos el algoritmo de Metropolis en el caso más simple posible: una distribución discreta con solo dos elementos, $a$ y $b$, con $P(a) = 2P(b)$. Las transiciones son de $b\rightarrow a$ siempre, y $a \rightarrow b$ o $a \rightarrow a$ al 50%. Por tanto, en términos de "media muestra", $b$ genera dos mitades de $a$ y $a$ genera media $a$ y media $b$. Para la proporción final da igual usar medias muestras que muestras completas.

In [ ]:
from collections import Counter

def samp(x):
    return x.replace('a','AB').replace('b','AA').lower()

s = 'b'
for k in range(10):
    s = samp(s)
    c = Counter(s)
    print(s)
    print(c, c['b']/c['a'])

Estas secuencias son los posibles estados finales después de 10 transiciones, todos ellos equiprobables. Si el sistema es ergódico el promedio temporal iguala al espacial, y por tanto la historia seguida es equivalente a los estados finales posibles.

Cuando las reglas de transformación se aplican con las proporciones que tiene cada elemento producen esas mismas proporciones: $1b,2a \rightarrow 2a, 2a,2b = 4a,2b$. Es la distribución estacionaria a la que tiende la cadena de Markov.

## Ejemplos de aplicación

### Media y varianza

Deseamos estimar la media y la dispersión de una variable normal con prioris no informativas, a partir de una pequeña muestra.

In [ ]:
data = np.random.randn(10)/2 + 1

print(np.mean(data),np.std(data))

data

En primer lugar suponemos conocida la dispersión.

In [ ]:
def lgauss1d(m, s, x):
    return -0.5 * ((x-m)/s)**2 - np.log(s)

def logprob(D):
    def f(θ):
        [m] = θ
        s = 0.5
        return sum(lgauss1d(m,s,D)) + 0
    return f

print(logprob(data)([0.8]))
print(logprob(data)([1]))

Ajustamos `sigma` para conseguir un $\rho$ alrededor de 0.3-0.4.

In [ ]:
test = metropolis(logprob(data), n=5000, burn=1000, step=3, sigma=0.5, x0=[0] )
h = plt.hist(test, bins=np.linspace(-3,3,40), color='lightgreen', edgecolor="gray")
plt.xlim(-3,3); plt.xlabel('media');

En segundo lugar suponemos conocida la media.

In [ ]:
def ljeffreys(s):
    return -np.log(s)

def logprob(D):
    def f(θ):
        [s] = θ
        if s <= 0: return -1e10
        m = 1
        return sum(lgauss1d(m,s,D)) + ljeffreys(s)
    return f

test = metropolis(logprob(data),n=5000, burn=1000, step=3, sigma=0.3, x0=[1])
h = plt.hist(test, bins=20, color='lightgreen', edgecolor="gray")
plt.xlim(0,3); plt.xlabel('desviación');

Y finalmente suponemos que tanto media como desviación son desconocidas.

In [ ]:
def logprob(D):
    def f(θ):
        m,s = θ
        if s <= 0: return -1e10
        return sum(lgauss1d(m,s,D)) + ljeffreys(s)
    return f

test = metropolis(logprob(data), n=5000, burn=1000, step=3, sigma=0.2, x0=[0,1] )
m,s = test.T
plt.figure(figsize=(6,6))
plt.plot(m,s,'.',markersize=2, alpha=0.5);
plt.xlabel('med'); plt.ylabel('std');

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
h = plt.hist(test[:,0], bins=20, color='lightgreen', edgecolor="gray")
plt.xlim(-3,3); plt.xlabel('mean')
plt.subplot(1,2,2)
h = plt.hist(test[:,1], bins=20, color='lightgreen', edgecolor="gray");
plt.xlabel('sigma');

Algunas de estas distribuciones a posteriori se pueden obtener de forma analítica. (Por ejemplo, la distribución de la media muestral cuando se desconoce la desviación es una *t-student*.) Pero en casos más generales solo podemos recurrir a técnicas computacionales. 

### Outliers

Veamos lo que ocurre si los datos están contaminados con *outliers*.

In [ ]:
noisydata = np.append(data,[-4,2,4])
noisydata

In [ ]:
test = metropolis(logprob(noisydata), n=5000, burn=1000, step=3, sigma=0.7, x0=[0,1] )
m,s = test.T
plt.figure(figsize=(6,6))
plt.plot(m,s,'.',markersize=2, alpha=0.5);

In [ ]:
%pip install -q https://raw.githubusercontent.com/albertoruiz/jupyterlite/main/content/misc/umucv-0.3-py3-none-any.whl

import umucv.prob as pr

def toProb(histo):
    h,b,_ = histo
    x = (b[:-1] + b[1:])/2
    return pr.P({x:h for x,h in zip(x,h)})

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
h = plt.hist(m, bins=20, color='lightgreen', edgecolor="gray")
pr.showhdi(toProb(h),95)
plt.xlim(-3,3); plt.xlabel('mean')
plt.subplot(1,2,2)
h = plt.hist(s, bins=20, color='lightgreen', edgecolor="gray")
pr.showhdi(toProb(h),95)
plt.xlim(0,3); plt.xlabel('sigma');

Lógicamente se obtienen estimaciones mucho más dispersas. Es necesario incorporar al modelo la posibilidad de que existan outliers, lo cual se puede hacer mediante un modelo de mezcla.

In [ ]:
def lunif(a,b,p):
    return -np.log(b-a) if a<=p<=b else -1e10

def gaussian1d(m,s,x):
    return 1/np.sqrt(2*np.pi)/s * np.exp ( -0.5 * ((x-m)/s)**2 )

def rmod(p,m,s,x):
    return np.log( (1-p)*gaussian1d(m ,s, x) + p* gaussian1d(0, 5, x) )

rmod(0.2,0,1,noisydata)

In [ ]:
def logprob(D):
    def f(θ):
        p,m,s = θ
        if s <= 0: return -1e10
        if not (0 <= p <= 1): return -1e8
        return sum(rmod(p,m,s,D)) + ljeffreys(s) + 0 + lunif(0,1,p)
    return f

logprob(noisydata)([0.2,0,1])

In [ ]:
logprob(noisydata)([0.3,1,0.5])

In [ ]:
test = metropolis(logprob(noisydata), n=5000, burn=1000, step=3, sigma=0.15, x0=[0.5,0,1])
p,m,s = test.T
plt.figure(figsize=(6,6))
plt.plot(m,s,'.',markersize=2, alpha=0.5);

In [ ]:
h = plt.hist(m,bins=30, color='lightblue');
pr.showhdi(toProb(h),95)
plt.xlim(-3,3);

In [ ]:
h = plt.hist(s,bins=30, color='lightblue');
pr.showhdi(toProb(h),95);

In [ ]:
h = plt.hist(p,bins=np.linspace(0,1,30), color='lightblue');
pr.showhdi(toProb(h),95)

Las estimaciones son ahora casi tan precisas como con datos limpios, y además tenemos una estimación sobre la proporción de outiliers en la muestra.

Lo más interesante es que si realizamos el mismo proceso con datos limpios, el método lo detecta dando una alta probabilidad a proporciones pequeñas de outiliers. La inferencia bayesiana controla automáticamente la capacidad del modelo.

In [ ]:
test = metropolis(logprob(data), n=5000, burn=1000, step=3, sigma=0.1, x0=[0.5,0,1])
p,m,s = test.T
plt.figure(figsize=(6,6))
plt.plot(m,s,'.',markersize=2, alpha=0.5);

In [ ]:
h = plt.hist(p,bins=np.linspace(0,1,20), color='orange');
pr.showhdi(toProb(h),95)

### Encuesta

¿Qué se puede decir de las proporciones reales de una cierta característica (por ejemplo, el porcentaje de votantes a unos partidos), a partir de una pequeña muestra?

Antes de empezar comprobamos que la forma de expresar las proporciones es correcta, sin favorecer a ninguna de ellas. Hay tres proporciones $p,q,r$ pero solo 2 grados de libertad ya que $r=1-p-q$.

In [ ]:
def logprob(p):
    p,q = p
    if p+q>1: return -1e10
    return lunif(0,1,p) + lunif(0,1,q)

test5 = metropolis(logprob, 5000, 1000, 3, 0.35, [0.2, 0.2])

In [ ]:
x,y = test5.T
plt.figure(figsize=(6,6))
plt.plot(x,y,'.',markersize=1); plt.axis('equal'); plt.axis([0,1,0,1]);

In [ ]:
np.mean(x), np.mean(y), np.mean(1-x-y)

In [ ]:
plt.hist(x,alpha=0.3,bins=20); plt.hist(y,alpha=0.3,bins=20); plt.hist(1-x-y, alpha=0.3,bins=20);

Ahora definimos el modelo de medida. Utilizamos el distribución multinomial, que nos dice la probabilidad de obtener $n_1,n_2,n_3$... éxitos de cada categoría cuando sus probabilidades son $p_1,p_2,p_3$... Es una generalización de la binomial.

In [ ]:
def lfact(n):
    return sum(np.log(np.arange(1,n+1)))

def lmultinom(ns, ps):
    return lfact(sum(ns)) - sum([lfact(n) for n in ns]) + sum([n*np.log(p) for n,p in zip(ns,ps)])

print(lmultinom([2,3,5],[0.2,0.4,0.4]))
print(lmultinom([2,3,5],[0.2,0.3,0.5]))
print(lmultinom([0,10,2],[0.2,0.3,0.5]))

En un primer experimento tenemos muy pocas respuestas, por lo que el resultado es bastante incierto.

In [ ]:
def logprob(D):
    def f(θ):
        p,q = θ
        if p+q>1 or min(p,q)<0: return -1e10
        return lmultinom(D, [p,q,1-p-q]) + lunif(0,1,p) + lunif(0,1,q)
    return f

test5 = metropolis(logprob([5,3,2]), 10000, 2000, 3, 0.2, [0.2, 0.2])

In [ ]:
x,y = test5.T
plt.figure(figsize=(6,6))
plt.plot(x,y,'.',markersize=2, alpha=0.2); plt.axis('equal'); 
plt.plot([0,1],[1,0],ls='dashed',color='gray',lw=0.5); plt.grid(ls='dotted');
plt.axis([0,1,0,1]);

La mismas proporciones en una muestra más amplia reducen la incertidumbre.

In [ ]:
test = metropolis(logprob([50,30,20]), 10000, 2000, 3, 0.06, [0.2, 0.2])

In [ ]:
x,y = test.T
plt.figure(figsize=(6,6))
plt.plot(x,y,'.',markersize=2, alpha=0.2); plt.axis('equal'); 
plt.plot([0,1],[1,0],ls='dashed',color='gray',lw=0.5); plt.grid(ls='dotted');
plt.axis([0,1,0,1]);

Ahora podemos evaluar la probabilidad de sucesos concretos:

In [ ]:
np.mean(x>0.45)

In [ ]:
np.mean(x>0.5)

In [ ]:
np.mean( y > (1-x-y) )

Podemos obtener estimaciones puntuales de los parámetros junto con su incertidumbre, pero no son independientes. Las estimaciones están centradas en el dato observado con 4-5% de desviación. Pero lógicamente no todos pueden estar a la vez en el extremo superior de su intervalo.

In [ ]:
np.mean(x), np.mean(y), np.mean(1-x-y)

In [ ]:
np.std(x), np.std(y), np.std(1-x-y)

### Billar raro

El problema que aparece en el [blog de Jake VanderPlas](http://jakevdp.github.io/blog/2014/06/06/frequentism-and-bayesianism-2-when-results-differ/).

In [ ]:
def logprob(p):
    [p] = p
    if not (0<=p<=1): return -1e10
    return lunif(0,1,p) + 3*np.log(p) + 5*np.log(1-p)

test4 = metropolis(logprob, 1000, 500, 10, 0.5, [0.5])

In [ ]:
np.mean(test4**3)

In [ ]:
h = plt.hist(test4,bins=np.linspace(0,1,20), color='lightgreen', edgecolor="gray")
pr.showhdi(toProb(h),95)

In [ ]:
from itertools import repeat

def bernoulli(p,a,b):
    return pr.P({a:p, b:1-p},norm=False)

seguir = (lambda b1: pr.joint(repeat(bernoulli(b1,0,1) ,3))) & toProb(h)
list(seguir.items())[:5]

In [ ]:
seguir.marginal(lambda s: sum(s[:3]))

In [ ]:
seguir.prob(lambda s: sum(s[:3])==0)